# Pembelajaran Mesin: Tugas Besar 1

Kaenova Mahendra Auditama  
IF-43-02  
1301190324  
  
Pada kodingan ini, saya mencoba untuk membuat sebuah model yang akan digunakan untuk membuat cluster pada suatu data. Data yang diberikan merupakan data ketertarikan pelanggan untuk membeli kendaraan baru. Pada akhirnya, saya menggunakan algoritma k-means karena kami diminta untuk membuat model dalam bentuk unsupervised learning
  
  
<sup>\*runs on AI Lab computer</sup>

# Import Library

In [2]:
%load_ext line_profiler
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pickle
import random
import math
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score, silhouette_samples
from urllib.request import urlopen
import multiprocessing as mp

# Modelling Before

In [3]:
class KMeansBefore:
  training_arr = None
  point = None
  inertia = None
  
  def __init__(self, df: pd.DataFrame):
    '''
    Kelas ini digunakan untuk menyiapkan dataframe yang akan ditraining.
    Pastikan kolom bernama id atau sejenis sudah di drop tidak termasuk ke dalam dataframe.
    '''
    print("K-Means akan ditentukan oleh atribut-atribut di bawah ini:")
    print("[", end="")
    for i in range(len(df.columns)):
      print(df.columns[i] + " ", end="")
    print("]", end="\n")
    self.training_arr = df.to_numpy()
    
  def fit_predict(self, k_num:int = 3, max_step:int = 500, conv_threshold: float = 1e-5) -> np.array:
    '''
    Membuat model KMeans dengan K tertentu. Akan mengkembalikan hasil prediksi cluster.
    Poin kluster akan disimpan pada variable point
    '''
    # Setting up cluster arry for every record
    cluster = np.zeros(len(self.training_arr))
    
    # normalize data
    data = self.__normalize_data__(self.training_arr)
    
    # Initialize centroid using KMeans++  
    point = self.__initialize_centroids__(data, k_num)
        
    # Setup convergence and counter
    convergence = False
    step = 0 
        
    while not convergence and (step < max_step):
      initial_point = point
      distance = self.__calculate_distance__(data, point)
      cluster = self.__clustering__(distance)
      new_point = self.__point_nomralization__(data, point, cluster)
      convergence = self.__convergence_check__(initial_point, new_point, conv_threshold)
      
      if convergence:
        point = new_point
        print("It's convergence!")
      else:
        point = new_point
        step += 1
        print("STEP:", step)
      
    
    self.inertia = self.__calculate_inertia__(data, cluster, point)
    self.point = self.__denormalize_point__(point, self.training_arr)
    return cluster
    
  # Made by Kaenova Mahendra Auditama | 1301190324 | IF-43-02
  def get_cluster_centroid(self) -> np.array:
    '''
    Fungsi ini digunakan untuk mengambil point
    '''
    if type(self.point) == "NoneType":
      print("Nothing returned, point not initialize. Try using fit_predict first.")
      return
    return self.point
  
  
  def __initialize_centroids__(self, data:np.array, k:np.array) -> np.array:
    '''
    Fungsi ini digunakan untuk menginisialisasikan centroid. Menggunakan algoritma k-means++
    referensi membantu: https://www.youtube.com/watch?v=HatwtJSsj5Q
    '''
    centroids = []
    random.seed(1) # To get same random result for benchmark purposes
    centroids.append( data[random.randrange(0, len(data))] )
    
    for i in range(1, k):
      min_dist = []
      for data_point in data:
        distance_data_point = []
        for point in centroids:
          distance_data_point.append(np.linalg.norm(data_point - point))
        min_dist.append(min(distance_data_point))
      
      probcum  = sum(min_dist)
      prob_point = [value / probcum for value in min_dist]
      
      centroids.append(data[np.argmax(prob_point)])
    
    return np.array(centroids)
  
  
  def __clustering__(self, distance: np.array) -> np.array:
    '''
    Fungsi ini akan mengembalikan hasil clustering berdasarkan distance
    '''
    cluster = np.zeros(len(distance))
    for i in range(len(cluster)):
      cluster[i] = np.argmin(distance[i])
    return cluster
  # Made by Kaenova Mahendra Auditama | 1301190324 | IF-43-02
  def __calculate_distance__(self, data:np.array, point: np.array) -> np.array:
    '''
    Fungsi ini akan menghitung setiap titik dengan point dan mengkembalikan jarak dari titik ke point
    '''
    distance = np.zeros((len(data), len(point)))
    
    for i in range(len(data)):
      current_record = data[i]
      for j in range(len(point)):
        current_point = point[j]
        # numpy eucledience distance
        distance[i][j] = np.linalg.norm(current_point - current_record)
    
    return distance
  
  def __point_nomralization__(self, data:np.array, point:np.array, cluster:np.array) -> (np.array, np.array):
    '''
    Fungsi ini digunakan untuk menghitung ulang kembali point dengan rata-rata
    '''
    new_point = np.zeros((len(point), len(point[0])))
    counter_array = np.zeros(len(point))
    for i in range(len(cluster)):
      new_point[int(cluster[i])] = new_point[int(cluster[i])] + data[i]
      counter_array[int(cluster[i])] += 1
      
    unique_on_cluster = np.unique(cluster)
    for i in range(len(point)):
      # nan handling
      if i not in unique_on_cluster:
        new_point[i] = point[i]
      else:
        new_point[i] = np.true_divide(new_point[i], counter_array[i])
      
    return new_point
  # Made by Kaenova Mahendra Auditama | 1301190324 | IF-43-02
  def __convergence_check__(self, points1: np.array, points2:np.array, threshold: float) -> bool:
    '''
    Fungsi ini untuk mengecek convergence berdasarkan threshold yang dibuat.
    titik cluster pertama akan dibandingkan dengan titik cluster kedua.
    note: maybe i should use euclediance distance insted of menghitung satu-satu
    '''
    local_convergence = False
    normalize_threshold_positive, normalize_threshold_negative  = 1 + threshold, 1 - threshold
    points_counter = 0
    center = np.zeros(len(points1[0]))
    for i in range(len(points1)):
      current_first_point, current_second_point = points1[i], points2[i]
      distance_first_point, distance_second_point = np.linalg.norm(current_first_point - center), np.linalg.norm(current_second_point - center)
      distance_threshold_positive = distance_first_point * normalize_threshold_positive
      distance_threshold_negative = distance_first_point * normalize_threshold_negative
      if distance_threshold_positive > distance_second_point and distance_threshold_negative < distance_second_point:
        points_counter += 1
    
    if points_counter == len(points1):
      local_convergence = True
    
    return local_convergence
  
  def __calculate_inertia__(self, data:np.array, cluster:np.array, points:np.array) -> np.array:
    '''
    K-Means: Inertia
    Inertia measures how well a dataset was clustered by K-Means. It is calculated by measuring the distance between each data point and its centroid, squaring this distance, and summing these squares across one cluster.
    ref: https://towardsdatascience.com/clustering-how-to-find-hyperparameters-using-inertia-b0343c6fe819
    '''
    inertia = 0
    for i in range(len(data)):
      inertia += (np.linalg.norm(data[i] - points[int(cluster[i])]))**2
      
    return inertia
  
  def __normalize_data__(self, data:np.array) -> np.array:
    '''
    Fungsi ini digunakan untuk menormalisasikan data dengan menggunakan min-max scaling. Sehingga data berjenis dan bersatuan apapun data diproses dengan baik.
    '''
    data = data.copy()
    for i in range(len(data[0])):
      col_arr = data[:,i]
      minmax = MinMaxScaler()
      normalize = minmax.fit_transform(col_arr.reshape(-1,1)).reshape(1,-1)
      data[:, i] = normalize[0]
      
    return data
  
  def __denormalize_point__(self, data:np.array, original_data:np.array) -> np.array:
    '''
    Fungsi ini digunakan untuk mendenormalisasikan point-point yang sudah dihitung menggunakan data yang ternormalisasi
    '''
    for i in range(len(data[0])):
      col_arr = data[:,i]
      col_arr_ori = original_data[:, i]
      
      minimums = min(col_arr_ori)
      maximums = max(col_arr_ori)
      for j in range(len(col_arr)):
        col_arr[j] = ((col_arr[j]*(maximums - minimums)) + minimums)
      data[:, i] = col_arr
    return data

# Modelling After

In [4]:
class KMeans:
  training_arr = None
  point = None
  inertia = None
  
  def __init__(self, df: pd.DataFrame):
    '''
    Kelas ini digunakan untuk menyiapkan dataframe yang akan ditraining.
    Pastikan kolom bernama id atau sejenis sudah di drop tidak termasuk ke dalam dataframe.
    '''
    print("K-Means akan ditentukan oleh atribut-atribut di bawah ini:")
    print("[", end="")
    for i in range(len(df.columns)):
      print(df.columns[i] + " ", end="")
    print("]", end="\n")
    self.training_arr = df.to_numpy()
    
  def fit_predict(self, k_num:int = 3, max_step:int = 500, conv_threshold: float = 1e-5) -> np.array:
    '''
    Membuat model KMeans dengan K tertentu. Akan mengkembalikan hasil prediksi cluster.
    Poin kluster akan disimpan pada variable point
    '''
    # Setting up cluster arry for every record
    cluster = np.zeros(len(self.training_arr))
    
    # normalize data
    data = self.__normalize_data__(self.training_arr)
    
    # Initialize centroid using KMeans++  
    point = self.__initialize_centroids__(data, k_num)
        
    # Setup convergence and counter
    convergence = False
    step = 0 
        
    while not convergence and (step < max_step):
      initial_point = point
      distance = self.__calculate_distance__(data, point)
      cluster = self.__clustering__(distance)
      new_point = self.__point_nomralization__(data, point, cluster)
      convergence = self.__convergence_check__(initial_point, new_point, conv_threshold)
      
      if convergence:
        point = new_point
        print("It's convergence!")
      else:
        point = new_point
        step += 1
        print("STEP:", step)
      
    
    self.inertia = self.__calculate_inertia__(data, cluster, point)
    self.point = self.__denormalize_point__(point, self.training_arr)
    return cluster
    
  # Made by Kaenova Mahendra Auditama | 1301190324 | IF-43-02
  def get_cluster_centroid(self) -> np.array:
    '''
    Fungsi ini digunakan untuk mengambil point
    '''
    if type(self.point) == "NoneType":
      print("Nothing returned, point not initialize. Try using fit_predict first.")
      return
    return self.point
  
  
  def __initialize_centroids__(self, data:np.array, k:np.array) -> np.array:
    '''
    Fungsi ini digunakan untuk menginisialisasikan centroid. Menggunakan algoritma k-means++
    referensi membantu: https://www.youtube.com/watch?v=HatwtJSsj5Q
    '''
    def _hitung_data_point_(num_process:int, data:np.array, centoids:np.array, return_dict:dict):
      min_dist = []
      for data_point in data:
        distance_data_point = []
        for point in centroids:
          distance_data_point.append(np.sum(((data_point-point)**2)**0.5) )
        min_dist.append(min(distance_data_point))
      return_dict[num_process] = min_dist
            
    centroids = []
    random.seed(1) # To get same random result for benchmark purposes
    centroids.append( data[random.randrange(0, len(data))] )
    
    for i in range(1, k):
      min_dist = []
      data_split = np.array_split(data, os.cpu_count())
      manager = mp.Manager()
      return_dict = manager.dict()
      processes = [mp.Process(target=_hitung_data_point_, args=(i, data_split[i], centroids, return_dict)) for i in range(len(data_split))]
      for process in processes:
        process.start()
      for process in processes:
        process.join()
      min_dist = [return_dict[i] for i in sorted(return_dict)]
      min_dist = np.concatenate((min_dist))
      
      probcum  = sum(min_dist)
      prob_point = [value / probcum for value in min_dist]
      
      centroids.append(data[np.argmax(prob_point)])
    
    return np.array(centroids)
  
  
  def __clustering__(self, distance: np.array) -> np.array:
    '''
    Fungsi ini akan mengembalikan hasil clustering berdasarkan distance
    '''
    cluster = np.zeros(len(distance))
    for i in range(len(cluster)):
      cluster[i] = np.argmin(distance[i])
    return cluster
  # Made by Kaenova Mahendra Auditama | 1301190324 | IF-43-02
  def __calculate_distance__(self, data:np.array, point: np.array) -> np.array:
    '''
    Fungsi ini akan menghitung setiap titik dengan point dan mengkembalikan jarak dari titik ke point
    '''
    # Setup penghitung
    def _hitung_data_point_(num_process:int, data:np.array, point:np.array, return_dict):
      distance = np.zeros((len(data), len(point)))
      for i in range(len(data)):
        current_record = data[i]
        for j in range(len(point)):
          current_point = point[j]
          distance[i][j] = np.sum(((current_record-current_point)**2)**0.5)   
      return_dict[num_process] = distance
      
    # Setup Processor dan bagi data
    data_split = np.array_split(data, os.cpu_count())
    manager = mp.Manager()
    return_dict = manager.dict()
    processes = [mp.Process(target=_hitung_data_point_, args=(i, data_split[i], point, return_dict)) for i in range(len(data_split))]
    
    # Run on all the process
    for process in processes:
      process.start()
    for process in processes:
      process.join()
    distance = [return_dict[i] for i in sorted(return_dict)]
    distance = np.concatenate((distance))
    return distance
  
  def __point_nomralization__(self, data:np.array, point:np.array, cluster:np.array) -> (np.array, np.array):
    '''
    Fungsi ini digunakan untuk menghitung ulang kembali point dengan rata-rata
    '''
    new_point = np.zeros((len(point), len(point[0])))
    counter_array = np.zeros(len(point))
    for i in range(len(cluster)):
      new_point[int(cluster[i])] = new_point[int(cluster[i])] + data[i]
      counter_array[int(cluster[i])] += 1
      
    unique_on_cluster = np.unique(cluster)
    for i in range(len(point)):
      # nan handling
      if i not in unique_on_cluster:
        new_point[i] = point[i]
      else:
        new_point[i] = np.true_divide(new_point[i], counter_array[i])
      
    return new_point
  # Made by Kaenova Mahendra Auditama | 1301190324 | IF-43-02
  def __convergence_check__(self, points1: np.array, points2:np.array, threshold: float) -> bool:
    '''
    Fungsi ini untuk mengecek convergence berdasarkan threshold yang dibuat.
    titik cluster pertama akan dibandingkan dengan titik cluster kedua.
    note: maybe i should use euclediance distance insted of menghitung satu-satu
    '''
    local_convergence = False
    normalize_threshold_positive, normalize_threshold_negative  = 1 + threshold, 1 - threshold
    points_counter = 0
    center = np.zeros(len(points1[0]))
    for i in range(len(points1)):
      current_first_point, current_second_point = points1[i], points2[i]
      distance_first_point, distance_second_point = np.sum(((current_first_point - center)**2)**0.5), np.sum(((current_second_point - center)**2)**0.5)
      distance_threshold_positive = distance_first_point * normalize_threshold_positive
      distance_threshold_negative = distance_first_point * normalize_threshold_negative
      if distance_threshold_positive > distance_second_point and distance_threshold_negative < distance_second_point:
        points_counter += 1
    
    if points_counter == len(points1):
      local_convergence = True
    
    return local_convergence
  
  def __calculate_inertia__(self, data:np.array, cluster:np.array, points:np.array) -> np.array:
    '''
    K-Means: Inertia
    Inertia measures how well a dataset was clustered by K-Means. It is calculated by measuring the distance between each data point and its centroid, squaring this distance, and summing these squares across one cluster.
    ref: https://towardsdatascience.com/clustering-how-to-find-hyperparameters-using-inertia-b0343c6fe819
    '''
    inertia = 0
    for i in range(len(data)):
      inertia += np.sum(((data[i]-points[int(cluster[i])])**2)**0.5)**2      
    return inertia
  
  def __normalize_data__(self, data:np.array) -> np.array:
    '''
    Fungsi ini digunakan untuk menormalisasikan data dengan menggunakan min-max scaling. Sehingga data berjenis dan bersatuan apapun data diproses dengan baik.
    '''
    data = data.copy()
    for i in range(len(data[0])):
      col_arr = data[:,i]
      minmax = MinMaxScaler()
      normalize = minmax.fit_transform(col_arr.reshape(-1,1)).reshape(1,-1)
      data[:, i] = normalize[0]
      
    return data
  
  def __denormalize_point__(self, data:np.array, original_data:np.array) -> np.array:
    '''
    Fungsi ini digunakan untuk mendenormalisasikan point-point yang sudah dihitung menggunakan data yang ternormalisasi
    '''
    for i in range(len(data[0])):
      col_arr = data[:,i]
      col_arr_ori = original_data[:, i]
      
      minimums = min(col_arr_ori)
      maximums = max(col_arr_ori)
      for j in range(len(col_arr)):
        col_arr[j] = ((col_arr[j]*(maximums - minimums)) + minimums)
      data[:, i] = col_arr
    return data

In [5]:
df_training = pd.read_csv("https://raw.githubusercontent.com/kaenova/Malin_Tubes1/main/data/processed/kendaraan_train_processed.csv")
df_training

,id,Umur,Kode_Daerah,Premi,Kanal_Penjualan,Lama_Berlangganan
0,1,30.0,33.0,28029.0,152.0,97.0
1,2,48.0,39.0,25800.0,29.0,158.0
2,4,58.0,48.0,2630.0,124.0,63.0
3,6,21.0,35.0,22735.0,152.0,171.0
4,9,20.0,8.0,30786.0,160.0,31.0
...,...,...,...,...,...,...
166391,285827,23.0,4.0,25988.0,152.0,217.0
166392,285828,21.0,46.0,44686.0,152.0,50.0
166393,285829,23.0,50.0,49751.0,152.0,226.0
166394,285830,68.0,7.0,30503.0,124.0,270.0


In [6]:
df_training = df_training.sample(100000)

# Before Optimizing

In [10]:
# No need to train again, i've already save the clusters
# But if you want to try the training part, just uncomment this
%load_ext line_profiler
MAX_K = 5

import pandas as pd

data_run_history = {
  "data" : "2 Data Kanal Penjualan dan Umur", 
  "k_runs" : {}
}

def run():
    for i in range(1, MAX_K+1):
      print("==== K:", i, "====" )
      run_data = {
        "points" : None,
        "cluster" : None,
        "inertia" : None
      }

      model = KMeansBefore(df_training.filter(items=["Umur","Kanal_Penjualan"]))
      run_data["cluster"] = model.fit_predict(k_num=i, conv_threshold=1e-15, max_step=4)
      run_data["inertia"] = model.inertia
      run_data["points"] = model.point
      data_run_history["k_runs"][i] = run_data
    
%lprun -f KMeansBefore.fit_predict run()

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
==== K: 1 ====
K-Means akan ditentukan oleh atribut-atribut di bawah ini:
[Umur Kanal_Penjualan ]
STEP: 1
It's convergence!
==== K: 2 ====
K-Means akan ditentukan oleh atribut-atribut di bawah ini:
[Umur Kanal_Penjualan ]
STEP: 1
STEP: 2
STEP: 3
STEP: 4
==== K: 3 ====
K-Means akan ditentukan oleh atribut-atribut di bawah ini:
[Umur Kanal_Penjualan ]
STEP: 1
STEP: 2
STEP: 3
STEP: 4
==== K: 4 ====
K-Means akan ditentukan oleh atribut-atribut di bawah ini:
[Umur Kanal_Penjualan ]
STEP: 1
STEP: 2
STEP: 3
STEP: 4
==== K: 5 ====
K-Means akan ditentukan oleh atribut-atribut di bawah ini:
[Umur Kanal_Penjualan ]
STEP: 1
STEP: 2
STEP: 3
STEP: 4


# After Optimizing

In [11]:
# No need to train again, i've already save the clusters
# But if you want to try the training part, just uncomment this
%load_ext line_profiler
MAX_K = 5

import pandas as pd

data_run_history = {
  "data" : "2 Data Kanal Penjualan dan Umur", 
  "k_runs" : {}
}

def run():
    for i in range(1, MAX_K+1):
      print("==== K:", i, "====" )
      run_data = {
        "points" : None,
        "cluster" : None,
        "inertia" : None
      }

      model = KMeans(df_training.filter(items=["Umur","Kanal_Penjualan"]))
      run_data["cluster"] = model.fit_predict(k_num=i, conv_threshold=1e-15, max_step=4)
      run_data["inertia"] = model.inertia
      run_data["points"] = model.point
      data_run_history["k_runs"][i] = run_data
    
%lprun -f KMeans.fit_predict run()

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
==== K: 1 ====
K-Means akan ditentukan oleh atribut-atribut di bawah ini:
[Umur Kanal_Penjualan ]
STEP: 1
It's convergence!
==== K: 2 ====
K-Means akan ditentukan oleh atribut-atribut di bawah ini:
[Umur Kanal_Penjualan ]
STEP: 1
STEP: 2
STEP: 3
STEP: 4
==== K: 3 ====
K-Means akan ditentukan oleh atribut-atribut di bawah ini:
[Umur Kanal_Penjualan ]
STEP: 1
STEP: 2
STEP: 3
STEP: 4
==== K: 4 ====
K-Means akan ditentukan oleh atribut-atribut di bawah ini:
[Umur Kanal_Penjualan ]
STEP: 1
STEP: 2
STEP: 3
STEP: 4
==== K: 5 ====
K-Means akan ditentukan oleh atribut-atribut di bawah ini:
[Umur Kanal_Penjualan ]
STEP: 1
STEP: 2
STEP: 3
STEP: 4
